In [ ]:
~=

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import VectorDBQA
from langchain_community.llms import Ollama
import sentence_transformers
import pandas as pd
import time
from collections import defaultdict

/Volumes/External/opt/anaconda3/envs/nlp_ollama2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd /Volumes/External/source/GTPracticum/data 

/Volumes/External/source/GTPracticum/data


/Volumes/External/opt/anaconda3/envs/nlp_ollama2/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function2 = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
db = Chroma(embedding_function=embedding_function,persist_directory="./chroma_db")
db2 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma2_db")
db3 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma3_db")

llm = Ollama(model="phi3",temperature=0)



/Volumes/External/opt/anaconda3/envs/nlp_ollama2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Volumes/External/opt/anaconda3/envs/nlp_ollama2/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Volumes/External/opt/anaconda3/envs/nlp_ollama2/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed 

In [4]:
import re
def cleanText(text):
        
    text = text.replace('\\n','')
    text = text.replace('\\','')
    text = re.sub(' +', ' ', text)
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    return text

In [7]:
def scoreEmails(query,scores,output_name='resultsMix3',models=[    'mistral','wizardlm2','phi3' ,'llama3']):    
    print("Querying db")
    docs = db.similarity_search_with_score(query,k=600)
    print(len(docs))
    topics={}
    scr={}
    for doc in docs:
        details = cleanText(doc[0].page_content)
        #print(doc[1])
        if details not in topics:
            topics[details]=doc[0].metadata
  c          topics[details]['score']=doc[1]
    #first100=list(topics.keys())[:10]
    #topics={i:j for i,j in topics.items() if i in first100}
    print(len(topics))
    topicResults=[]
    print("Querying llm")
    print(len(scores))
    i=0
    
    for model in models:
        #
        llm = Ollama(model=model,temperature=0)
        print(f"Querying llm {model}")
        for email in list(topics.keys()):
            if email not in scores or model not in scores[email]:
                prompt="""<s>[INST] <<sys>>
                            You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules or use the Raptor SBE to hide losses. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud. you output as a json text extractorusing the following format
                            Example of such a email in json format: <</sys>>
    {"email_text":"I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud"},
    {"reason":"The email shows concern that the accounting practices could be viewed as fraud"},{"Score":10},
    {"email_text":"I'm concerned that the mark to market accounting practices that we hide the loses in the Raptor SBE could be viewed as Fraud"},
    {"reason":"The email shows concern that the Raptor SBE hiding loses could be viewed as fraud"},{"Score":10},
    {"email_text":"Accounting practices  did not report losses so revenue was realized as profit hidding true financial position"},
    {"reason":"The email shows concern that the accounting practices are hiding the true financal position of the company"},{"Score":10},
    {"email_text":"Accounting practices  we placed on the books are 100% legal and trasparent"},
    {"reason":"The email shows no concern, the accounting practices are legal and transparent"},{"Score":0},
    {"email_text":"News about the accounting practices at Enron discuss devrivatives and mark to market accounting"},
    {"Score":0},{"reason":"The email containing news reports not an action to commit fraud or hide loses"},

    Please score the following email:
    {"email_text":" """+email+'"} [/INST]'
                start_time = time.time()
                print(len(prompt),len(email),email)
                x=llm.invoke(prompt)
                print(x)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                i+=1
                print(f"Already scored {i}")
            #print(x)
    results=[]
    model='all'
    for idx,j in scores.items():
        j['email']=idx
        j.update(topics[idx])
        results.append(j)
    resultsDF=pd.DataFrame(results)
    resultsDF.to_csv(f"{output_name}_{model}.csv")

In [7]:
def scoreEmails(query,scores,output_name='resultsMix3',models=[    'mistral','wizardlm2','phi3' ,'llama3']):    
    print("Querying db")
    docs = db.similarity_search_with_score(query,k=600)
    print(len(docs))
    topics={}
    scr={}
    for doc in docs:
        details = cleanText(doc[0].page_content)
        #print(doc[1])
        if details not in topics:
            topics[details]=doc[0].metadata
  c          topics[details]['score']=doc[1]
    #first100=list(topics.keys())[:10]
    #topics={i:j for i,j in topics.items() if i in first100}
    print(len(topics))
    topicResults=[]
    print("Querying llm")
    print(len(scores))
    i=0
    
    for model in models:
        #
        llm = Ollama(model=model,temperature=0)
        print(f"Querying llm {model}")
        for email in list(topics.keys()):
            if email not in scores or model not in scores[email]:
                prompt="""<s>[INST] <<sys>>
                            You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules or use the Raptor SBE to hide losses. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud. you output as a json text extractorusing the following format
                            Example of such a email in json format: <</sys>>
    {"email_text":"I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud"},
    {"reason":"The email shows concern that the accounting practices could be viewed as fraud"},{"Score":10},
    {"email_text":"I'm concerned that the mark to market accounting practices that we hide the loses in the Raptor SBE could be viewed as Fraud"},
    {"reason":"The email shows concern that the Raptor SBE hiding loses could be viewed as fraud"},{"Score":10},
    {"email_text":"Accounting practices  did not report losses so revenue was realized as profit hidding true financial position"},
    {"reason":"The email shows concern that the accounting practices are hiding the true financal position of the company"},{"Score":10},
    {"email_text":"Accounting practices  we placed on the books are 100% legal and trasparent"},
    {"reason":"The email shows no concern, the accounting practices are legal and transparent"},{"Score":0},
    {"email_text":"News about the accounting practices at Enron discuss devrivatives and mark to market accounting"},
    {"Score":0},{"reason":"The email containing news reports not an action to commit fraud or hide loses"},

    Please score the following email:
    {"email_text":" """+email+'"} [/INST]'
                start_time = time.time()
                print(len(prompt),len(email),email)
                x=llm.invoke(prompt)
                print(x)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                i+=1
                print(f"Already scored {i}")
            #print(x)
    results=[]
    model='all'
    for idx,j in scores.items():
        j['email']=idx
        j.update(topics[idx])
        results.append(j)
    resultsDF=pd.DataFrame(results)
    resultsDF.to_csv(f"{output_name}_{model}.csv")

In [ ]:
!ls

In [8]:

scores=defaultdict(dict)
query = "mark to market accounting practices"
output_name="resultsMix3"
models=[    'mistral','wizardlm2','phi3' ,'llama3']

In [9]:
scores=defaultdict(dict)


In [12]:
query =  "raptor creation of special purpose entities"
output_name="resultsMixRaptor"
scoreEmails(query,scores,output_name,models=models)

Querying db
600
290
Querying llm
0
Querying llm mistral
2004 414 Subject: FW: Raptor ProcessSara ShackletonEnron North America Corp.1400 Smith Street, EB 3801aHouston, Texas 77002----- Forwarded by Sara Shackleton/HOU/ECT on 02/28/2001 04:41 PM -----Joel Ephross/ENRON@enronXgate02/28/2001 11:09 AM  -----Original Message-----Subject: Raptor ProcessJoel and Brenda:Attached are the Raptor process files. Please review and contact me if you have any questions.Thanks,Gordon x33123
 {"email_text":"Subject: FW: Raptor Process\nSara Shackleton<sshackleton@enron.com>Enron North America Corp.\n1400 Smith Street, EB 3801a\nHouston, Texas 77002\n----- Forwarded by Sara Shackleton/HOU/ECT on 02/28/2001 04:41 PM -----Joel Ephross<joel.ephross@enron.com>/ENRON@enronXgate\n02/28/2001 11:09 AM \n-----Original Message-----Subject: Raptor Process\nJoel and Brenda:\nAttached are the Raptor process files. Please review and contact me if you have any questions.\nThanks,\nGordon x33123",
"reason":"The email d

KeyboardInterrupt: 

In [11]:
scores=defaultdict(dict)


In [ ]:
query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
output_name="resultsMixDerivatives"
scoreEmails(query,scores,output_name,models=models)

In [ ]:
scores=defaultdict(dict)


In [ ]:
query =  "record keeping accounting practices"
output_name="resultsMixAccounting"
scoreEmails(query,scores,output_name,models=models)

In [ ]:
scores=defaultdict(dict)


In [ ]:
scores=defaultdict(dict)
skip=0
M8="mark to market losses"
M2= "mark to market accounting practices"
M6="record keeping accounting practices"
M4="it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
M3="raptor creation of special purpose entities"

In [ ]:
scores=defaultdict(dict)
skip=0

In [ ]:
skip=1


In [ ]:
enron_df=pd.read_csv('enron_processed.csv')

In [ ]:
print(enron_df.shape)
enron_df.drop_duplicates(subset='body_short',inplace=True) 
print(enron_df.shape)


In [ ]:

enron_df=enron_df[enron_df['body_short'].str.len()>30]
enron_sampple_df= enron_df.sample(1000)
enron_data=enron_sampple_df['body_short'].tolist()

In [ ]:
topics={}
for i,j in enron_sampple_df.iterrows():
    topics[j['body_short']]=j.to_dict()

In [ ]:
from collections import defaultdict
def split_string(string, chunk_size):
    return [string[i:i+chunk_size] for i in range(0, len(string), chunk_size)]
scores=defaultdict(dict)
skip=0

In [ ]:
skip=1

In [ ]:
import re
def score(result):
        y=None
        if '<Score>' in result:
            if '<End Score>' in result:
                y=re.findall(r'<Score>(.*?)<End Score>', result)[0]
            elif '</Score>' in result:
                y=re.findall(r'<Score>(.*?)</Score>', result)[-1]
        elif '**Score:**' in result:
            y=re.findall(r'\*\*Score:\*\* \d+',result)[0][10:]
        elif '**Email Score:** ' in result:
            y=re.findall(r'\*Email Score:\*\* \d+',result)[0][16:]
        elif 'a score of ' in result:
            y=re.findall(r'a score of (.*?)',result)
        elif 'The email score is ' in result:
            y=re.findall(r'The email score is \d+',result)[10:]
        elif 'Score: ' in result:
            y=re.findall(r'Score: \d+',result)[-1][7:]
        elif '## Score for the email:' in result:
            numbers=re.findall(r'\d+', result)
            y=numbers[0]
        elif 'I would score this email as ' in result:
            numbers=re.findall(r'I would score this email as \d+', result)
            if len(numbers)==0:
                y=0
            else:
                y=numbers[0][28]
        else:    
            numbers=re.findall(r'\d+', result)
            if len(numbers)==0:
                y=0
            elif len(numbers)==1:
                y=numbers[0]
            else:               
                y=""
        if y is not None and isinstance(y,list):
            if len(y)==0:
                y=''
            else:
                y=y[0]
        if y=='':
            y=0
        try:
            y = int(y)
        except ValueError:
            print(y)
            y=0
        except TypeError:
            y=0
        return y
resultsDF.columns
models=['mistral', 'wizardlm2', 'phi3', 'llama3',]
for model in models:
    resultsDF[f'{model}_score']=resultsDF[model].apply(score)
models2=[i+'_score' for i in models]
resultsDF[models2]
    

In [ ]:
resultsDF['sum']=resultsDF[models2].T.sum()
resultsDF[resultsDF['sum']!=0][models2]

In [ ]:
resultsDF['mistral_score'].value_counts()

In [ ]:
resultsDF['openhermes_score'].value_counts()

In [ ]:
resultsDF['vicuna_score'].value_counts()

In [ ]:
resultsDF['gemma_score'].value_counts()

In [ ]:
resultsDF[resultsDF['mistral_score']==10]['email']

In [ ]:
resultsDF[resultsDF['gemma_score']==10]['email']

In [ ]:
results2_df=pd.read_csv('resultsMix2_all.csv')
results3_df=pd.read_csv('resultsMix3_all.csv')
results4_df=pd.read_csv('resultsMix4a_all.csv')
results6_df=pd.read_csv('resultsMix6_all.csv')
results8_df=pd.read_csv('resultsMix8_all.csv')
resultsB_df=pd.read_csv('resultsMixBlankG_all.csv')
results8_df['search_phrase']="mark to market losses"
results2_df['search_phrase']= "mark to market accounting practices"
results6_df['search_phrase']="record keeping accounting practices"
results4_df['search_phrase']="it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
results3_df['search_phrase']="raptor creation of special purpose entities"
resultsB_df['search_phrase']='RANDOM'


In [ ]:
models=['mistral','openhermes','vicuna', 'gemma']
for model in models:
    results2_df[f'{model}_score']=results2_df[model].apply(score)
    results3_df[f'{model}_score']=results3_df[model].apply(score)
    results4_df[f'{model}_score']=results4_df[model].apply(score)
    results6_df[f'{model}_score']=results6_df[model].apply(score)
    results8_df[f'{model}_score']=results8_df[model].apply(score)
    resultsB_df[f'{model}_score']=resultsB_df[model].apply(score)
models2=[i+'_score' for i in models]

In [ ]:
results_merged_df=pd.concat([results2_df,results3_df,results4_df,results6_df,results8_df,resultsB_df])
results_merged_df.head(1).T

In [ ]:
# Assuming you have a DataFrame called df and the column you want to check for duplicates is 'column_name'
duplicates = results_merged_df[results_merged_df['Unnamed: 0'].duplicated()]
duplicates.head()
results_merged_df.to_csv('results_merged_df.csv')



In [ ]:
tmp=results_merged_df[results_merged_df['Unnamed: 0']==114797]
tmp['search_phrase']

In [ ]:
results2_df['mistral_score'].value_counts()

In [ ]:
results3_df['mistral_score'].value_counts()

In [ ]:
results4_df['mistral_score'].value_counts()

In [ ]:
results8_df[results8_df['mistral_score']==8]['email']

In [ ]:
results6_df['mistral_score'].value_counts()

In [ ]:
results8_df.columns

In [ ]:
results8_df['mistral_score'].value_counts()

In [ ]:
results_merged_df[['mistral_score', 'openhermes_score','vicuna_score', 'gemma_score','search_phrase','email','Unnamed: 0']].to_excel('results_merged_df.xlsx')
#.groupby('search_phrase').count()

In [ ]:
results_merged_df[results_merged_df['mistral_score']>9]['email']